In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import copy



In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
# local imports
from predicament.utils.file_utils import load_featured_dataframe_and_config
from predicament.utils.file_utils import write_featured_dataframe_and_config


In [5]:
# input parameters
core_subdir = 'dreem_4secs' # dataset
# norm_subdir = 'baseline_'+core_subdir # dataset of standardising statistics
norm_subdir = core_subdir # dataset of standardising statistics
new_subdir = core_subdir + '_standardised'
std_group = 'participant'

print(f"core_subdir = {core_subdir}")
print(f"norm_subdir = {norm_subdir}")
print(f"new_subdir = {new_subdir}")

core_subdir = dreem_4secs
norm_subdir = dreem_4secs
new_subdir = dreem_4secs_standardised


## Load  data
Both the core data and the data for deriving the statistics for standardisation.


In [6]:
core_df, core_config = load_featured_dataframe_and_config(
    core_subdir)
norm_df, norm_config = load_featured_dataframe_and_config(
    norm_subdir)


In [7]:
print(f"core_config:\n{core_config}\n\n")
print(f"norm_config:\n{norm_config}\n\n")

core_config:
{'DEFAULT': {}, 'LOAD': {'participant_list': ['VG_01', 'VG_02', 'VG_03', 'VG_05', 'VG_06', 'VG_07', 'VG_08', 'VG_09', 'VG_10', 'VH_01', 'VH_02', 'VH_03'], 'conditions': ['exper_video', 'wildlife_video', 'familiar_music', 'tchaikovsky', 'family_inter'], 'channels': ['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG Fpz-F8'], 'n_channels': 4, 'sample_rate': 250, 'label_mapping': ['exper_video', 'familiar_music', 'family_inter', 'tchaikovsky', 'wildlife_video'], 'data_format': 'dreem'}, 'WINDOWED': {'group_col': 'participant', 'target_col': 'condition', 'label_cols': ['participant', 'condition', 'window index'], 'window_size': 1024, 'window_step': 128}, 'FEATURED': {'feature_set': ['SD', 'IQR', 'SampleEntropy', 'Hurst', 'Min', 'Mean', 'Correlation', 'arCoeff', 'MeanFreq', 'LempelZivComplexity', 'FreqKurtosis', 'LempelZivEntropy', 'Energy', 'MAD', 'FreqSkewness', 'Max', 'LyapunovExponent', 'MaxFreqInd'], 'feature_names': ['LempelZivComplexity[b=1]', 'SD2', 'arCoeff10', 'Correlati

In [8]:
MUST_BE_EQUAL = {}
MUST_BE_EQUAL['LOAD'] = [
    'data_format', 'n_channels', 'channels',
    'participant_list', 'sample_rate'
]
MUST_BE_EQUAL['WINDOWED'] = [
    'window_size', 'window_step', 'group_col']


def check_data_compatible(
        core_config, norm_config, must_be_equal=MUST_BE_EQUAL):
    for section in must_be_equal.keys():
        for term in must_be_equal[section]:
            core_val = core_config[section][term]
            norm_val = norm_config[section][term]
            type_ = type(core_val)
            err_msg = f"config[{section}][{term}] not equal"
            assert type_ == type(norm_val)
            if type_ is list:
                assert np.all([
                    e1 == e2 for e1, e2 in zip(core_val, norm_val)]), err_msg
            elif type_ is float:
                np.testing.assert_almost_equal(
                    core_val, norm_val,
                    err_msg=err_msg)
            else:
                assert core_val == norm_val, err_msg

                

In [9]:
check_data_compatible(core_config, norm_config)

In [10]:
core_feature_set = core_config['FEATURED']['feature_set']
core_feature_names = core_config['FEATURED']['feature_names'] 
core_feature_names.sort()
print(f"core_feature_names: {core_feature_names}")
norm_feature_set = norm_config['FEATURED']['feature_set']
norm_feature_names = norm_config['FEATURED']['feature_names'] 
norm_feature_names.sort()
print(f"norm_feature_names: {norm_feature_names}")

new_feature_set = list(
    set(core_feature_set).intersection(set(norm_feature_set)))
new_feature_set.sort()
new_feature_names = list(
    set(core_feature_names).intersection(set(norm_feature_names)))
new_feature_names.sort()

dropped_feature_names = list(
    set(core_feature_names) - set(new_feature_names))
dropped_feature_names.sort()
print(f"new_feature_set: {new_feature_set}")
print(f"new_feature_names: {new_feature_names}")
print(f"dropped_feature_names: {dropped_feature_names}")

new_df = core_df
new_df.drop(columns=dropped_feature_names, inplace=True)


core_feature_names: ['Correlation0', 'Correlation1', 'Correlation2', 'Correlation3', 'Correlation4', 'Correlation5', 'Energy0', 'Energy1', 'Energy2', 'Energy3', 'FreqKurtosis0', 'FreqKurtosis1', 'FreqKurtosis2', 'FreqKurtosis3', 'FreqSkewness0', 'FreqSkewness1', 'FreqSkewness2', 'FreqSkewness3', 'Hurst_C0', 'Hurst_C1', 'Hurst_C2', 'Hurst_C3', 'Hurst_H0', 'Hurst_H1', 'Hurst_H2', 'Hurst_H3', 'IQR0', 'IQR1', 'IQR2', 'IQR3', 'LempelZivComplexity[b=1]', 'LempelZivComplexity[b=2]', 'LempelZivComplexity[b=4]', 'LempelZivComplexity[b=8]', 'LempelZivEntropy[b=1][0]', 'LempelZivEntropy[b=1][1]', 'LempelZivEntropy[b=1][2]', 'LempelZivEntropy[b=1][3]', 'LempelZivEntropy[b=4][0]', 'LempelZivEntropy[b=4][1]', 'LempelZivEntropy[b=4][2]', 'LempelZivEntropy[b=4][3]', 'LyapunovExponent0', 'LyapunovExponent1', 'LyapunovExponent2', 'LyapunovExponent3', 'MAD0', 'MAD1', 'MAD2', 'MAD3', 'Max0', 'Max1', 'Max2', 'Max3', 'MaxFreqInd0', 'MaxFreqInd1', 'MaxFreqInd2', 'MaxFreqInd3', 'Mean0', 'Mean1', 'Mean2', 'Mea

In [11]:
new_df['participant'].unique()

array(['VG_01', 'VG_02', 'VG_03', 'VG_05', 'VG_06', 'VG_07', 'VG_08',
       'VG_09', 'VG_10', 'VH_01', 'VH_02', 'VH_03'], dtype=object)

In [12]:
norm_df['participant'].unique()

array(['VG_01', 'VG_02', 'VG_03', 'VG_05', 'VG_06', 'VG_07', 'VG_08',
       'VG_09', 'VG_10', 'VH_01', 'VH_02', 'VH_03'], dtype=object)

In [13]:
norm_df = norm_df.loc[:,[std_group]+new_feature_names]

# Calculate the mean and standard deviation for each group in df
mean_df = norm_df.groupby(std_group).mean().reset_index()
std_df = norm_df.groupby(std_group).std().reset_index()

# self_standardised_features = ['HRVRMSSD[Normed]']
# to_self_standardise = list(
#     set(self_standardised_features).intersection(
#         set(dropped_feature_names)))
# for ssf  in to_self_standardise:
#     tmp_df = new_df[['participant',ssf]]
#     mean = tmp_df.groupby(
#         'participant').mean().reset_index()
#     std = tmp_df.groupby(
#         'participant').std().reset_index()
#     mean_df[ssf] = mean
#     std_df[ssf] = std

# new_feature_names.extend(to_self_standardise)    
# dropped_feature_names = list(
#     set(core_feature_names) - set(new_feature_names))
# dropped_feature_names.sort()
# print(f"new_feature_set: {new_feature_set}")
# print(f"new_feature_names: {new_feature_names}")
# print(f"dropped_feature_names: {dropped_feature_names}")

print("Mean DataFrame:")
print(mean_df)

print("\nStandard Deviation DataFrame:")
print(std_df)


# Merge the mean and std data frames with new_df on 'participant'
new_df = new_df.merge(mean_df, on=std_group, suffixes=('', '_mean'))
new_df = new_df.merge(std_df, on=std_group, suffixes=('', '_std'))

# standardise the data columns
dropped_columns = []
for col in new_feature_names:
#     print(f"col = {col}")
    new_df[col] = (new_df[col] - new_df[col + '_mean']) / new_df[col + '_std']
    dropped_columns.append(col + '_mean')
    dropped_columns.append(col + '_std')

new_df.drop(columns=dropped_columns, inplace=True)
    

Mean DataFrame:
   participant  Correlation0  Correlation1  Correlation2  Correlation3  \
0        VG_01      0.416371      0.476369      0.040664      0.142180   
1        VG_02      0.618519      0.875133      0.783268      0.438816   
2        VG_03      0.523860      0.321925     -0.002162      0.155433   
3        VG_05      0.414641      0.057226      0.102577      0.038978   
4        VG_06      0.139917     -0.036184      0.209268     -0.037422   
5        VG_07      0.403612      0.174780      0.030629      0.062062   
6        VG_08      0.053946      0.068781      0.255571     -0.182758   
7        VG_09      0.725158      0.216513      0.126458      0.128028   
8        VG_10      0.157625      0.076887      0.055820      0.165956   
9        VH_01      0.145285      0.124469      0.026162      0.011797   
10       VH_02      0.722193      0.247236      0.222250      0.187373   
11       VH_03      0.781503      0.276526      0.039807      0.227147   

    Correlation4  Cor

In [14]:
new_config = copy.deepcopy(core_config)

new_config['FEATURED']['feature_set'] = new_feature_set
new_config['FEATURED']['feature_names'] = new_feature_names

new_config['STANDARDISED'] = {}
new_config['STANDARDISED']['core_subdir'] = core_subdir
new_config['STANDARDISED']['norm_subdir'] = norm_subdir
new_config['STANDARDISED']['std_group'] = std_group

write_featured_dataframe_and_config(
    new_subdir, new_df, new_config)


writing data to ../data/featured/dreem_4secs_standardised/featured.csv
writing config to ../data/featured/dreem_4secs_standardised/details.cfg
